In [1]:
def count(g,s):
    TP = TN = FP = FN = 0
    for i in range(0,len(g)):
        if (g[i]==s[i] and s[i]==1): TP+=1
        if (g[i]==s[i] and s[i]==0): TN+=1
        if (g[i]!=s[i] and s[i]==1): FP+=1
        if (g[i]!=s[i] and s[i]==0): FN+=1
    return [TP,TN,FP,FN]
    
def MSRP_eval(gs, sys):
    [TP,TN,FP,FN] = count(gs,sys)
    acc = (TP+TN)/float(TP+TN+FP+FN) # ACCURACY
    reject = TN/float(TN+FP) # precision on negative SPECIFICITY
    accept = TP/float(TP+FN) # precision on positive SENSITIVITY
    print("acc=",acc," reject=",reject," accept=",accept)

In [100]:
import numpy as np
import pandas as pd
import nbimporter

from nltk.parse.bllip import BllipParser
from nltk.data import find
from nltk import Tree
from nltk.draw.util import CanvasFrame
from nltk.draw import TreeWidget
from sklearn.linear_model import LogisticRegression
from nltk.metrics import *
from nltk.tag import *
from nltk.corpus import treebank
from sent_utils import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
init_notebook_mode(connected=True)
MODEL_PATH = find('models/bllip_wsj_no_aux').path
PARSER = BllipParser.from_unified_model_dir(MODEL_PATH)

In [103]:
# Paraphrase detectors
def bllip_paraphrase_detector(sent_0, sent_1):
    print('.', end='')
    sent_0, sent_1 = remove_punctuation(sent_0.lower()), remove_punctuation(sent_1.lower())
    words_0, words_1 = words_from_sent(sent_0), words_from_sent(sent_1)
    tokens_0, tokens_1 = tokens_from_words(words_0), tokens_from_words(words_1)
    lemmas_0, lemmas_1 = tokens_to_lemmas(tokens_0), tokens_to_lemmas(tokens_1)
    senses_0, senses_1 = tokens_to_senses(tokens_0), tokens_to_senses(tokens_1)
    top_synsets_0, top_synsets_1 = [sense[0] for sense in senses_0], [sense[0] for sense in senses_1]
#     synset_distance = []
#     for synsetA in top_synsets_0:
#         for synsetB in top_synsets_1:
#             similarity = synsetA.path_similarity(synsetB)
#             if similarity:
#                 synset_distance.append(similarity)
    return [
        jaccard_distance(set(top_synsets_0), set(top_synsets_1)),
        jaccard_distance(set(lemmas_0), set(lemmas_1))
#         ,
#         np.mean(synset_distance)
    ]
def bllip_parse():
    PARSER.parse_one(line.split())
def draw_tree(line):
    cf = CanvasFrame()
    t = Tree.fromstring(line)
    tc = TreeWidget(cf.canvas(),t)
    cf.add_widget(tc,10,10) # (10,10) offsets
    cf.print_to_file('tree.ps')
    cf.destroy()

In [7]:
input_sentences = text_to_sentences('IHLT-eval-framework/train/msr_paraphrase_train_input.txt')

In [8]:
X_train = [lemma_synset_jaccard(data[0], data[1]) for data in input_sentences]

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [9]:
y_train = open('IHLT-eval-framework/train/msr_paraphrase_train_gs.txt', encoding="UTF8").readlines()
y_train[0] = '1'
y_train = [int(line.strip()) for line in y_train]

In [10]:
X_test = text_to_sentences('IHLT-eval-framework/test/msr_paraphrase_test_input.txt')
X_test = [lemma_synset_jaccard(data[0], data[1]) for data in X_test]

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [24]:
y_test = open('IHLT-eval-framework/test/msr_paraphrase_test_gs.txt', encoding="UTF8").readlines()
y_test = [int(line.strip()) for line in y_test]

In [12]:
regression = LogisticRegression()
regression.fit(X_train, y_train)
prediction = regression.predict(X_test)
MSRP_eval(prediction, y_test)

acc= 0.7043478260869566  reject= 0.5982658959537572  accept= 0.7309644670050761


In [86]:
X_test = text_to_sentences('IHLT-eval-framework/test/msr_paraphrase_test_input.txt')
false_negative_idx = np.where((prediction != y_test) & (prediction == 0))[0]

In [99]:
false_negatives = [ X_test[i] for i in list(indices[0]) ]
lengths = []
for n in false_negatives:
    if (len(n[0]) == 44):
        print(n)
    lengths.append(len(n[0]))

['Shares ended Wednesday at $6.83, up 2 cents.', 'Shares of Goodyear rose 2 cents on Wednesday and closed at $6.83.']


In [98]:
np.min(lengths)

44